In [1]:
#imports
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import callbacks
from keras import optimizers

Using TensorFlow backend.


In [2]:
DEV = False
argvs = sys.argv
argc = len(argvs)

if argc > 1 and (argvs[1] == "--development" or argvs[1] == "-d"):
  DEV = True

if DEV:
  epochs = 12
else:
    # default epoches which get trained when you dont start the file via terminal
  epochs = 100

train_data_path = './data/train'
validation_data_path = './data/validation'

"""
Parameters
"""
img_width, img_height = 150, 150
batch_size = 64
samples_per_epoch = 5400
validation_steps = 2100
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 8
lr = 0.0004 # learning rate from 0.5 till -0.5 take care with the learning rate check keras optimizer documentary

In [4]:
model = Sequential()

#added layers, per .add a layer gets added

model.add(Conv2D(nb_filters1, (conv1_size, conv1_size), padding ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Conv2D(nb_filters2, (conv2_size, conv2_size), padding ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format='channels_first'))

model.add(Flatten())
model.add(Dense(1024)) # dense model connections increased dense model from 256 to 1024
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

# compile your model with a specific loss function there a lot of loss function choose the best one for your model
# we want loss to decrease in our training
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'],)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 9311 images belonging to 8 classes.
Found 9311 images belonging to 8 classes.


In [ ]:
#start the training with .fit_generator
model.fit_generator(
    train_generator,
    steps_per_epoch=samples_per_epoch // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps // batch_size)

target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)

#saving the trained model and weights of the models 
model.save('./models/modelv4.1_8.categories.h5')
model.save_weights('./models/weights_v4.1_8.categories.0.h5')

In [ ]:
#predict_multiclass
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model


img_width, img_height = 150, 150
model_path = './models/modelv4.0_8.categories.h5'
model_weights_path = './models/weights_v4.0_8.categories.0.h5'
model = load_model(model_path)
model.load_weights(model_weights_path)

# label the prediction with the names of the specific categories
def predict(file):
  x = load_img(file, target_size=(img_width,img_height))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = model.predict(x)
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:
    print("Label: banana")
  elif answer == 1:
    print("Labels: battery")
  elif answer == 2:
    print("Label: computer")
  elif answer == 3:
    print("Label: glass_bottel")
  elif answer == 4:
    print("Label: light_blub")
  elif answer == 5:
    print("Label: paper")
  elif answer == 6:
    print("Label: phone")
  elif answer == 7:
    print("Label: plastic")
  

  return answer

In [ ]:
# classify a picture with "result" what is a label array which includes the specific category label
for i, ret in enumerate(os.walk('./test-data/whatishere')):
  for i, filename in enumerate(ret[2]):
    result = predict(ret[0] + '/' + filename)